In [1]:
!wget https://hydranets-data.s3.eu-west-3.amazonaws.com/hydranets-data.zip && unzip -q hydranets-data.zip && mv hydranets-data/* . && rm hydranets-data.zip && rm -rf hydranets-data

--2024-10-11 10:51:09--  https://hydranets-data.s3.eu-west-3.amazonaws.com/hydranets-data.zip
Resolving hydranets-data.s3.eu-west-3.amazonaws.com (hydranets-data.s3.eu-west-3.amazonaws.com)... 3.5.225.205, 3.5.225.182
Connecting to hydranets-data.s3.eu-west-3.amazonaws.com (hydranets-data.s3.eu-west-3.amazonaws.com)|3.5.225.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 110752264 (106M) [application/zip]
Saving to: ‘hydranets-data.zip’

hydranets-data.zip  100%[===================>] 105.62M  17.4MB/s    in 7.4s    

2024-10-11 10:51:17 (14.3 MB/s) - ‘hydranets-data.zip’ saved [110752264/110752264]



In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cv2
import torch
import torch.nn as nn
import math
import torch.nn.functional as F
from torch.autograd import Variable

### Encoder

In [19]:
class HydraNet(nn.Module):
    def __init__(self):
        #super(HydraNet, self).__init__() # Python2
        super().__init__() # Python 3
        self.num_tasks = 2
        self.num_classes = 6

In [20]:
hydranet = HydraNet()

In [21]:
def conv3x3(in_channels, out_channels, stride=1, dilation=1, groups=1, bias=False):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=dilation, bias=bias, groups=groups)

In [22]:
def conv1x1(in_channels, out_channels, stride=1, dilation=0, groups=1, bias=False):
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=dilation, bias=bias, groups=groups)

In [23]:
def batchnorm(num_features):
    return nn.BatchNorm2d(num_features, affine=True, eps=1e-5, momentum=0.1)

In [24]:
def convbnrelu(in_channels, out_channels, kernel_size, stride=1, groups=1, act=True):
  if act:
    return nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=int(kernel_size/2.0), groups=groups, bias=False),
                         batchnorm(out_channels),
                         nn.ReLU6(inplace=True))
  else:
    return nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=int(kernel_size/2.0), groups=groups, bias=False),
                         batchnorm(out_channels))

In [25]:
class InvertedResidualBlock(nn.Module):
  def __init__(self, in_planes, out_planes, expansion_factor, stride=1):
    super().__init__()
    intermed_planes = in_planes * expansion_factor
    self.residual = (in_planes == out_planes) and (stride == 1)
    self.output = nn.Sequential(convbnrelu(in_planes, intermed_planes, 1),
                                convbnrelu(intermed_planes, intermed_planes, 3, stride=stride, groups=intermed_planes),
                                convbnrelu(intermed_planes, out_planes, 1, act=False))
  def forward(self, x):
    if self.residual:
      return x + self.output(x)
    else:
      return self.output(x)

In [26]:
def define_mobilenet(self):
  mobilenet_config = [[1, 16, 1, 1],
                      [6, 24, 2, 2],
                      [6, 32, 3, 2],
                      [6, 64, 4, 2],
                      [6, 96, 3, 1],
                      [6, 160, 3, 2],
                      [6, 320, 1, 1]]

  self.in_channels = 32
  self.num_layers = len(mobilenet_config)
  self.layers1 = convbnrelu(3, self.in_channels, kernel_size=3, stride=2)
  c_layer = 2

  for t, c, n, s in mobilenet_config:
    layers = []
    for idx in range(n):
      layers.append(InvertedResidualBlock(self.in_channels, c, expansion_factor=t, stride=s if idx == 0 else 1))
      self.in_channels = c
    setattr(self, "layer{}".format(c_layer), nn.Sequential(*layers))
    c_layer += 1

HydraNet.define_mobilenet = define_mobilenet

In [27]:
hydranet.define_mobilenet()

In [28]:
print(hydranet)

HydraNet(
  (layers1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace=True)
  )
  (layer2): Sequential(
    (0): InvertedResidualBlock(
      (output): Sequential(
        (0): Sequential(
          (0): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Sequential(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (2): Sequential(
          (0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affi

### Decoder

In [29]:
class CRPBlock(nn.Module):
  def __init__(self, in_planes, out_planes, n_stages, group=False):
    super().__init__()
    for i in range(n_stages):
      setattr(self, "{}_{}".format(i+1, "outvar_dimred"),
              conv1x1(in_planes if (i == 0) else out_planes, out_planes,
                      stride=1, bias=False,
                      groups=in_planes if group else 1))
    self.stride = 1
    self.n_stages = n_stages
    self.maxpool = nn.MaxPool2d(kernel_size=5, stride=1, padding=2)
  def forward(self, x):
    top = x
    for i in range(self.n_stages):
      top = self.maxpool(top)
      top = getattr(self, "{}_{}".format(i+1, "outvar_dimred"))(top)
      x = top + x
    return x

In [30]:
def _make_crp(self, in_planes, out_planes, stages, group=False):
  layers = [CRPBlock(in_planes, out_planes, stages, group)]
  return nn.Sequential(*layers)

In [31]:
HydraNet._make_crp = _make_crp

In [35]:
def define_lightweight_refinenet(self):
  self.conv8 = conv1x1(320, 256, bias=False)
  self.conv7 = conv1x1(160, 256, bias=False)
  self.conv6 = conv1x1(96, 256, bias=False)
  self.conv5 = conv1x1(64, 256, bias=False)
  self.conv4 = conv1x1(32, 256, bias=False)
  self.conv3 = conv1x1(24, 256, bias=False)
  self.crp4 = self._make_crp(256, 256, 4, group=False)
  self.crp3 = self._make_crp(256, 256, 4, group=False)
  self.crp2 = self._make_crp(256, 256, 4, group=False)
  self.crp1 = self._make_crp(256, 256, 4, group=False)

  self.conv_adapt4 = conv1x1(256, 256, bias=False)
  self.conv_adapt3 = conv1x1(256, 256, bias=False)
  self.conv_adapt2 = conv1x1(256, 256, bias=False)

  self.pre_depth = conv1x1(256, 256, groups=256, bias=False)
  self.depth = conv3x3(256, 1, bias=True)
  self.pre_segm = conv1x1(256, 256, groups=256, bias=False)
  self.segm = conv3x3(256, self.num_classes, bias=True)
  self.relu = nn.ReLU(inplace=True)

  if self.num_tasks == 3:
    self.pre_normal = conv1x1(256, 256, groups=256, bias=False)
    self.normal = conv3x3(256, 3, bias=True)

In [36]:
HydraNet.define_lightweight_refinenet = define_lightweight_refinenet

In [37]:
hydranet.define_lightweight_refinenet()

### Forward

In [38]:
def forward(self, x):
  x = self.layers1(x)
  x = self.layer2(x)
  l3 = self.layer3(x)
  l4 = self.layer4(l3)
  l5 = self.layer5(l4)
  l6 = self.layer6(l5)
  l7 = self.layer7(l6)
  l8 = self.layer8(l7)

  l8 = self.conv8(l8)
  l7 = self.conv7(l7)
  l7 = self.relu(l8 + l7)
  l7 = self.crp4(l7)
  l7 = self.conv_adapt4(l7)
  l7 = nn.Upsample(size=l6.size()[2:], mode="bilinear", align_corners=False)(l7)

  self.conv6(l6)
  self.conv5(l5)
  self.relu(l5 + l6 + l7)
  self.crp3(l5)
  self.conv_adapt3(l5)
  nn.Upsample(size=l4.size()[2:], mode="bilinear", align_corners=False)(l5)

  self.conv4(l4)
  self.relu(l5 + l4)
  self.crp2(l4)
  self.conv_adapt2(l4)
  nn.Upsample(size=l3.size()[2:], mode="bilinear", align_corners=False)(l4)

  self.conv3(l3)
  self.relu(l3 + l4)
  self.crp1(l3)

  # Heads

  out_segm = self.segm(l3)
  out_segm = self.relu(out_segm)
  out_segm = self.segm(out_segm)

  out_d = self.depth(l3)
  out_d = self.relu(out_d)
  out_d = self.depth(out_d)

  if self.num_tasks == 3:
    out_n = self.pre_normal(l3)
    out_n = self.relu(out_n)
    out_n = self.normal(out_n)
    return out_segm, out_d, out_n
  else:
    return out_segm, out_d

In [39]:
HydraNet.forward = forward